In [3]:
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 11.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 16.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.0 MB/s eta 0:00:00
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53725 sha256=a640c8cb22e7dda060205408f94d3c3e376f0721c5b790495e55a9b957249975
  Stored in directory: /Users/seokjoongkim/Library/Caches/pip/wh

In [87]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ['OPENAI_API_KEY'] = 

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [88]:
user_query = 'agent memory'

In [89]:
import bs4
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(texts, embeddings)

vector_store_retrievers = [vectorstore.as_retriever(), vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
), vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
), vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
), vectorstore.as_retriever(search_kwargs={'k': 1}), vectorstore.as_retriever(
    search_kwargs={'filter': {'paper_title': 'GPT-4 Technical Report'}}
)]


Created a chunk of size 2731, which is longer than the specified 1000
Created a chunk of size 1538, which is longer than the specified 1000
Created a chunk of size 1380, which is longer than the specified 1000
Created a chunk of size 2352, which is longer than the specified 1000
Created a chunk of size 1953, which is longer than the specified 1000
Created a chunk of size 1067, which is longer than the specified 1000
Created a chunk of size 1475, which is longer than the specified 1000
Created a chunk of size 2881, which is longer than the specified 1000
Created a chunk of size 1980, which is longer than the specified 1000
Created a chunk of size 4145, which is longer than the specified 1000
Created a chunk of size 2159, which is longer than the specified 1000
Created a chunk of size 1317, which is longer than the specified 1000
Created a chunk of size 1112, which is longer than the specified 1000
Created a chunk of size 1043, which is longer than the specified 1000
Created a chunk of s

In [96]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

parser = JsonOutputParser()

relevant_prompt = PromptTemplate(
    template="Determine if the query is relevant to docs.\n{format_instructions}\n{query}\n{docs}",
    input_variables=["query", "docs"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = relevant_prompt | llm | parser

relevant_retrievers = []
irrelevant_retrievers = []

for retriever in vector_store_retrievers:
    docs = retriever.invoke(user_query)
    res = chain.invoke({"query": user_query, "docs": docs})
    print(res)
    if res.get('relevant', False):
        relevant_retrievers.append(retriever)
    else:
        irrelevant_retrievers.append(retriever)



{'relevant': True}
{'relevant': True, 'explanation': "The query 'agent memory' is directly relevant to the documents, particularly the first document which discusses long-term memory modules for agents, their observations, and the context of retrieval models. Other documents also touch on related themes such as task execution and generative agent architecture, which are relevant to the concept of 'agent memory'."}
{'relevant': True, 'reason': "The query 'agent memory' is directly relevant to the documents provided, particularly the first document, which discusses the concept of memory in agents, including long-term memory modules, retrieval models, and reflection mechanisms."}


/Users/seokjoongkim/llm/joon.233/myenv/lib/python3.12/site-packages/langchain_core/vectorstores/base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'relevant': False, 'message': "The query 'agent memory' does not provide enough context to determine relevance to specific documents."}
{'relevant': True}
{'relevant': False, 'message': 'The query does not provide enough context to determine relevance to the documents.'}


In [97]:
relevant_checker = 'I like an apple'

In [98]:
validated_retrievers = []

for retriever in relevant_retrievers:
    docs = retriever.invoke(relevant_checker)
    res = chain.invoke({"query": relevant_checker, "docs": docs})
    if not res.get('relevant', True):
        validated_retrievers.append(retriever)

print(validated_retrievers)

[VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127b97080>), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127b97080>, search_type='mmr', search_kwargs={'k': 6, 'lambda_mult': 0.25}), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127b97080>, search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 50}), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127b97080>, search_kwargs={'k': 1})]


In [100]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [102]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [109]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = None
for retriever in validated_retrievers:
    docs = retriever.invoke(user_query)
    
    cur_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
    )
    if rag_chain is None:
        rag_chain = cur_chain
    else:
        rag_chain = rag_chain | cur_chain




Agent memory is a long-term module that records experiences in natural language, allowing for the retention and recall of information. It enables agents to synthesize past observations into higher-level inferences, guiding future actions. The retrieval model prioritizes recent, relevant, and important memories to enhance agent behavior and communication.


In [115]:
if rag_chain is not None:
    hallucination_checker_prompt = PromptTemplate(
        template="Determine if the query has hallucination or not.\n{format_instructions}\n{query}",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    hallucination_chain = hallucination_checker_prompt | llm | parser
    
    possible_answer = 'No'
    has_hallucination = True
    while has_hallucination:
        
        answer_list = []
        for chunk in rag_chain.stream(user_query):
            answer_list.append(chunk)
    
        possible_answer = "".join(answer_list)
        checker = hallucination_chain.invoke({"query": possible_answer})
        if not checker.get('hallucination', True):
            break
        else:
            print(checker)
            print('fail to pass hallucination check')
    
    answer = possible_answer
    print(answer)
else:
    print('No')

    

{'has_hallucination': False, 'reason': 'The query accurately describes the components of agent memory, including short-term and long-term memory, without introducing false or misleading information.'}
fail to pass hallucination check
Agent memory comprises both short-term and long-term components, with long-term memory often stored in an external database for vast information retention. This structure enhances interaction by allowing the agent to recall accumulated knowledge when responding. Additionally, the agent can utilize external APIs to supplement its existing knowledge with current and specific information.
